In [10]:
import fetch_rewards as fr
import pandas as pd

In [11]:
brands_df = fr.load_brands_data()
users_df = fr.load_users_data()
receipts_df = fr.load_receipts_data()

# Checking data quality before Relational Data Model

### Brands data

#### Null values
The most important information like barcode, name and id are non null here. But all other information has null values that makes it difficult for us to categorize the brands and make better decisions

In [2]:
print(brands_df.isnull().sum())

brands_df.info()


barcode           0
category        155
categoryCode    650
name              0
topBrand        612
_id.$oid          0
cpg.$id.$oid      0
cpg.$ref          0
brandCode       234
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       1167 non-null   object
 1   category      1012 non-null   object
 2   categoryCode  517 non-null    object
 3   name          1167 non-null   object
 4   topBrand      555 non-null    object
 5   _id.$oid      1167 non-null   object
 6   cpg.$id.$oid  1167 non-null   object
 7   cpg.$ref      1167 non-null   object
 8   brandCode     933 non-null    object
dtypes: object(9)
memory usage: 82.2+ KB


#### Duplicate barcodes

Usually we expect unique entries for each barcode. If there are duplicates that indicates same item has different entries

In [3]:
# find duplicates in barcode
duplicates = brands_df[brands_df.duplicated(subset=['barcode'], keep=False)]

print(len(duplicates["barcode"].unique()))

# Check for duplicates
print("\nTotal Duplicate entries in the dataframe:")
print(brands_df[brands_df.duplicated()])

7


#### Barcode validity

we can check the barcode validity by making some assumptions that the barcodes are numeric - do not include letters or any special characters and they have a certain length in this case 12

Following cell shows that all the barcodes are valid

In [12]:
# Check for invalid barcodes (assuming valid barcodes are numeric and of a certain length)
print("\nInvalid barcodes (non-numeric or incorrect length):")
valid_barcode_length = 12 
invalid_barcodes = brands_df[~brands_df['barcode'].astype(str).str.isnumeric() | (brands_df['barcode'].astype(str).str.len() != valid_barcode_length)]
print(invalid_barcodes)


Invalid barcodes (non-numeric or incorrect length):
Empty DataFrame
Columns: [barcode, category, categoryCode, name, topBrand, _id.$oid, cpg.$id.$oid, cpg.$ref, brandCode]
Index: []


### Users data

#### Null Values

Identifier columns like user_id, created_date and role do not have any null values. This is promising for the data

In [13]:
print(users_df.info())
# check for null values
print(users_df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   active          212 non-null    bool          
 1   role            212 non-null    object        
 2   sign_up_source  207 non-null    object        
 3   state           206 non-null    object        
 4   user_id         212 non-null    object        
 5   created_date    212 non-null    datetime64[ns]
 6   last_login      172 non-null    datetime64[ns]
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 10.3+ KB
None
active             0
role               0
sign_up_source     5
state              6
user_id            0
created_date       0
last_login        40
dtype: int64


#### Duplicates

There are also no duplicates in user_id column

In [15]:
# check for duplicate userIds
duplicates = users_df[users_df.duplicated(subset=['user_id'], keep=False)]
print(len(duplicates["user_id"].unique()))

0


#### Checking validity of date columns

Any created, modified, login dates should not be later than current timestamp
Login date should not be earlier than created date


In [17]:
# check if created_data and last_login are later than current date
users_df['created_date'] = pd.to_datetime(users_df['created_date'])
users_df['last_login'] = pd.to_datetime(users_df['last_login'])
print(users_df[users_df['created_date'] > pd.Timestamp.now()])
print(users_df[users_df['last_login'] > pd.Timestamp.now()])

# check if login date is earlier than created date
print(users_df[users_df['last_login'] < users_df['created_date']])

Empty DataFrame
Columns: [active, role, sign_up_source, state, user_id, created_date, last_login]
Index: []
Empty DataFrame
Columns: [active, role, sign_up_source, state, user_id, created_date, last_login]
Index: []
Empty DataFrame
Columns: [active, role, sign_up_source, state, user_id, created_date, last_login]
Index: []


This is by far the cleanest data we got with very few null values and usage of well defined data types. Although the original json can be made more efficient with reducing nested structures for date and user_id, which we can easily cleanup in a python script

### Receipts data

#### Null values
Identifier columns like _id, user_id and decision making column rewards_receipt_status are non-null which allows us to use the data for parts of our decision making. From the relational model user_id will be a foreign key from users table so that's a good sign

Other columns have more than 50% null values which is concerning. We can make some transformations as shown below

In [5]:
print(receipts_df.isnull().sum())
receipts_df.info()

bonusPointsEarned          575
bonusPointsEarnedReason    575
pointsEarned               510
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
_id.$oid                     0
createDate.$date             0
dateScanned.$date            0
finishedDate.$date         551
modifyDate.$date             0
pointsAwardedDate.$date    582
purchaseDate.$date         448
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bonusPointsEarned        544 non-null    float64
 1   bonusPointsEarnedReason  544 non-null    object 
 2   pointsEarned             609 non-null    object 
 3   purchasedItemCount       635 non-null    float64
 4   rewardsReceiptItemList   679 non-null    object 
 5   rewardsReceiptStatus     1119 no

#### Duplicates

Receipt ids should not be duplicated

In [18]:
# check for duplicates
duplicates = receipts_df[receipts_df.duplicated(subset=['_id.$oid'], keep=False)]

print(len(duplicates["_id.$oid"].unique()))

0


#### Understanding points columns

Points_earned and bonus_points_earned should be float columns.

Assuming points_earned Column is the total points Earned, we don't have any columns that have bonus_pointed_earned greater than points_earned.

Points earned should not be negative

In [19]:
receipts_df_temp = receipts_df

# Assuming PointsEarned column is the total points earned by the user
# if the value is null update it with BonusPointsEarned
receipts_df_temp['pointsEarned'] = receipts_df_temp['pointsEarned'].fillna(receipts_df_temp['bonusPointsEarned'])

# convert pointsEarned column to float
receipts_df_temp['pointsEarned'] = receipts_df_temp['pointsEarned'].astype(float)

# number of rows with BonusPointsEarned > PointsEarned
print("Number of rows with BonusPointsEarned > PointsEarned:")
print(len(receipts_df_temp[receipts_df_temp['bonusPointsEarned'] > receipts_df_temp['pointsEarned']]))

# check for negative values in pointsEarned
print("Number of rows with negative pointsEarned:")
print(len(receipts_df[receipts_df['pointsEarned'] < 0]))

# check for negative values in bonusPointsEarned
print("Number of rows with negative bonusPointsEarned:")
print(len(receipts_df[receipts_df['bonusPointsEarned'] < 0]))


Number of rows with BonusPointsEarned > PointsEarned:
0
Number of rows with negative pointsEarned:
0
Number of rows with negative bonusPointsEarned:
0


#### Check validity of dates columns

Any date columns should not be later than current time stamp

modify_date should not be earlier than create_date

finished_date and points_awarded_date should not be earlier than create_date and date_scanned

purchase_date should not be later than date_scanned. There are 13 rows with purchase date later than date scanned.

**Note:** Not all scenarios are covered in the code below. We are just trying to get the basic understanding of the date columns



In [24]:
# modify createDate.$date to datetime format
receipts_df['create_date'] = pd.to_datetime(receipts_df['createDate.$date'], unit='ms')
# check if createDate is later than current date
print("Number of rows with createDate later than current date:")
print(len(receipts_df[receipts_df['create_date'] > pd.Timestamp.now()]))

# modify purchaseDate.$date to datetime format
receipts_df['purchase_date'] = pd.to_datetime(receipts_df['purchaseDate.$date'], unit='ms')
# check if purchaseDate is later than current date
print("Number of rows with purchaseDate later than current date:")
print(len(receipts_df[receipts_df['purchase_date'] > pd.Timestamp.now()]))

# modify datescanned.$date to datetime format
receipts_df['date_scanned'] = pd.to_datetime(receipts_df['dateScanned.$date'], unit='ms')
# check if dateScanned is later than current date
print("Number of rows with dateScanned later than current date:")
print(len(receipts_df[receipts_df['date_scanned'] > pd.Timestamp.now()]))

# check if purchaseDate is later than dateScanned
print("Number of rows with purchaseDate later than date_scanned:")
print(len(receipts_df[receipts_df['purchase_date'] > receipts_df['date_scanned']]))

# modify finishedDate.$date to datetime format
receipts_df['finished_date'] = pd.to_datetime(receipts_df['finishedDate.$date'], unit='ms')
# check if finishedDate is later than current date
print("Number of rows with finishedDate later than current date:")
print(len(receipts_df[receipts_df['finished_date'] > pd.Timestamp.now()]))

# check if finishedDate is earlier than dateScanned, createdate
print("Number of rows with finishedDate earlier than dateScanned or createdate:")
print(len(receipts_df[(receipts_df['finished_date'] < receipts_df['date_scanned']) | (receipts_df['finished_date'] < receipts_df['create_date'])]))


Number of rows with createDate later than current date:
0
Number of rows with purchaseDate later than current date:
0
Number of rows with dateScanned later than current date:
0
Number of rows with purchaseDate later than date_scanned:
13
Number of rows with finishedDate later than current date:
0
Number of rows with finishedDate earlier than dateScanned or createdate:
0


Data Quality exploration in the rest of the notebook is on extracted data as we see in the Entity relationship model. There might be slight differences in the results when compared with using the raw data from JSON files. Assuming our long term vision is to store the data in a structured format in a data warehouse we want to understand the data quality of the modeled data 

### Receipt Item list

We cannot do any duplicate checks on the receipt_item_list table as we do not expect any unique identifiers in this table. Just by checking the info of the dataframe it is clear that most columns have significant null values. 

More than half of the receipts are missing product_id that makes it difficult for us to match them with the product and brands tables

In [25]:
receipt_item_list = fr.create_receipt_items_table(receipts_df)
print(receipt_item_list.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 24 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   receipt_id                             6941 non-null   object 
 1   product_id                             3090 non-null   object 
 2   final_price                            6767 non-null   object 
 3   needs_fetch_review                     813 non-null    object 
 4   partner_item_id                        6941 non-null   object 
 5   prevent_target_gap_points              358 non-null    object 
 6   quantity_purchased                     6767 non-null   float64
 7   user_flagged_barcode                   337 non-null    object 
 8   user_flagged_new_item                  323 non-null    object 
 9   user_flagged_price                     299 non-null    object 
 10  user_flagged_quantity                  299 non-null    float64
 11  need

#### Understanding Purchased Item count

We are expecting the count of purchased items from top level receipts table to match the total of quantity purchased on from receipt_items_list table. Out of 1119 receipts 533 do not have matching item count. This could be because of null values in the receipt_item_list table or because the app read the values wrong when the receipt is scanned.

In [29]:
# check total_quantity_purchased per receipt_id is same as item_count in receipts_df
# group by receipt_id and sum total_quantity_purchased
grouped_df = receipt_item_list.groupby('receipt_id')['quantity_purchased'].sum().reset_index()
# merge with receipts_df to compare with item_count
receipts_df_grouped = receipts_df.merge(grouped_df, right_on='receipt_id', left_on="_id.$oid", how='left', suffixes=('', '_sum'))
print("Number of rows with total_quantity_purchased not equal to item_count:")
print(len(receipts_df_grouped[receipts_df_grouped['quantity_purchased'] != receipts_df_grouped['purchasedItemCount']]))


Number of rows with total_quantity_purchased not equal to item_count:
533


# Conclusion

* We have obtained high-quality user data due to the implementation of a standardized and fixed input mechanism.
* Brands and products cannot be categorized as expected in the Entity Relation model due to null values, which prevent the establishment of proper unique identifiers for these tables. To resolve this data quality issue, it is recommended to separate user-facing data from backend payer data. Columns such as metabrite_id, payer_id, and partner_item_id can be stored in a separate table, while efforts should be made to improve user-facing information like product_id, brand_name, and user_flags to avoid null values.
* Fortunately, there are no duplicates in the tables where it matters the most.
* It is important to enforce strict rules on the date columns. For example, the purchase date should not be later than the date scanned. Our data contains 13 records that violate this rule.
* An interesting observation is that the total purchased item count does not match between the high-level receipts table and the receipt_items_list. This discrepancy could be resolved by improving the model for receipt scanning.
